## Import Libraries

In [3]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.onnx
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

## Load Data

In [4]:
os.chdir("..")

In [5]:
os.getcwd()

'd:\\Projects\\AI_projects\\e2e_github'

In [6]:
df = pd.read_csv(r"data\Titanic-Dataset.csv")

## Preproccessing

In [7]:
df = df[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked", "Survived"]]
df = df.dropna()

## Encode Categorical variables

In [8]:
df["Sex"] = LabelEncoder().fit_transform(df["Sex"])
df["Embarked"] = LabelEncoder().fit_transform(df["Embarked"])

## Features and Labels

In [9]:
x = df.drop("Survived", axis=1).values
y = df["Survived"].values

## Train Test Split

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [11]:
x_train.shape

(569, 7)

## Build Dataset

In [12]:
class TitanicDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)   
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    
train_ds = TitanicDataset(x_train, y_train)
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)

## Build Model

In [13]:
class TitanicModel(nn.Module):
    def __init__(self):
        super(TitanicModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(x_train.shape[1], 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.net(x)
    
model = TitanicModel()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Train

In [14]:
for epoch in range(100):
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(x_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item(): 4f}")

Epoch 1, Loss:  0.765218
Epoch 2, Loss:  0.594184
Epoch 3, Loss:  0.680883
Epoch 4, Loss:  0.429023
Epoch 5, Loss:  0.656018
Epoch 6, Loss:  0.820642
Epoch 7, Loss:  0.568831
Epoch 8, Loss:  0.482989
Epoch 9, Loss:  0.436573
Epoch 10, Loss:  0.692035
Epoch 11, Loss:  0.559253
Epoch 12, Loss:  0.461448
Epoch 13, Loss:  0.815061
Epoch 14, Loss:  0.597040
Epoch 15, Loss:  0.597840
Epoch 16, Loss:  0.433972
Epoch 17, Loss:  0.416503
Epoch 18, Loss:  0.725736
Epoch 19, Loss:  0.645563
Epoch 20, Loss:  0.681057
Epoch 21, Loss:  0.708795
Epoch 22, Loss:  0.553554
Epoch 23, Loss:  0.587530
Epoch 24, Loss:  0.478481
Epoch 25, Loss:  0.333081
Epoch 26, Loss:  0.605488
Epoch 27, Loss:  0.493680
Epoch 28, Loss:  0.425360
Epoch 29, Loss:  0.381100
Epoch 30, Loss:  0.284647
Epoch 31, Loss:  0.374374
Epoch 32, Loss:  0.409824
Epoch 33, Loss:  0.498373
Epoch 34, Loss:  0.560754
Epoch 35, Loss:  0.319230
Epoch 36, Loss:  0.864308
Epoch 37, Loss:  0.409215
Epoch 38, Loss:  0.444398
Epoch 39, Loss:  0.48

## Save Model

In [15]:
torch.save(model.state_dict(), "models/titanic_model.pth")
print("Model saved as titanic_model.pth")

Model saved as titanic_model.pth


## Using onnx to save the model

In [17]:
dummy_input = torch.randn(1, x_train.shape[1])

torch.onnx.export(
    model,
    dummy_input,
    "models/titanic_model.onnx",
    export_params = True,
    opset_version= 11,
    do_constant_folding= True,
    input_names = ["input"],
    output_names = ["output"],
    dynamic_axes= {"input": {0: "batch_size"}, "output": {0: "batch_size"}}
)

print("Model saved as titanic_model.onnx")

Model saved as titanic_model.onnx
